### Load packages 

In [2]:

import os
import glob

project_dir = "/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/"
#project_dir = "/standard/vol328/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/"
#project_dir ="/Users/zqn7td/Desktop/9samples_cellstate/"

os.chdir("/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/Scripts")

In [3]:
##Import packages


import anndata as ad
import scimap as sm
import importlib

import scanpy as sp
import pandas as pd
import numpy as np
import napari as napari
import math
import seaborn as sns
import itertools

from scipy import stats
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import colors

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, leaves_list
from scipy.spatial.distance import pdist
from matplotlib.colors import Normalize

import hetmel as hm

Running SCIMAP  2.3.5


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [4]:
os.chdir('/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/Scripts')
importlib.reload(hm)

<module 'hetmel' from '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/Scripts/hetmel.py'>

## Set Case number here

In [72]:
case_numbers = [
    "Case1-31",
    "Case2-8",
    
    "Case3-9",
    "Case4-5_R1",
    "Case4-5_R2",
    "Case7-10",
    "Case8-9_R1",
    "Case8-9_R2",
    "Case9-7_R2",
    "Case10-3_R1",
    "Case10-3_R2",
    "Case11-13"
]

phenotype = pd.read_csv(os.path.join(project_dir, '20251002_phenotyping_workflow_fullpanel.csv'))

## Generate list containing normalized adata of all cases

find paths to gates.csv files 

In [8]:

gate_file_paths = {}

for case_number in case_numbers:
    # Search for gates.csv inside the case-specific subdirectory
    search_path = os.path.join(project_dir, case_number, '*gates.csv')
    matches = glob.glob(search_path)
    if matches:
        gate_file_paths[case_number] = matches[0]  # Take the first match
        print(f"Found for {case_number}: {matches[0]}")
    else:
        gate_file_paths[case_number] = None
        print(f"No gates.csv file found for {case_number}")

Found for Case1-31: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv
Found for Case2-8: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case2-8/Case2-8_full_gates.csv
Found for Case3-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_gates.csv
Found for Case4-5_R1: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R1/Case4-5_R1_full_gates.csv
Found for Case4-5_R2: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R2/Case4-5_R2_full_gates.csv
Found for Case7-10: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case7-10/Case7-10_full_gates.csv
Found for Case8-9_R1: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/2

In [9]:
gate_file_paths

{'Case1-31': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv',
 'Case2-8': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case2-8/Case2-8_full_gates.csv',
 'Case3-9': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_gates.csv',
 'Case4-5_R1': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R1/Case4-5_R1_full_gates.csv',
 'Case4-5_R2': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R2/Case4-5_R2_full_gates.csv',
 'Case7-10': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case7-10/Case7-10_full_gates.csv',
 'Case8-9_R1': '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Cas

Loop through adata for each case, normalize the adata to that case's gate values, and append normalized data to a list

In [11]:
adata_list = []

for case_number in case_numbers:
    if gate_file_paths[case_number] is not None:
        print(f"Reading in raw adata for {case_number}")
        # Read in raw adata file for this case
        adata_path = os.path.join(project_dir, 'raw_adata', f'{case_number}_raw.h5ad')
        adata = ad.read_h5ad(adata_path)
        
        print(f"Copying gates from {gate_file_paths[case_number]}")
        marker_gates = pd.read_csv(os.path.join( project_dir, case_number, f'{case_number}_full_gates.csv'),
        header = [0], index_col =[0])
        adata.uns['gates'] = marker_gates
        
        hm.normalize_to_gates(adata)
        # Copy the original layer
        adata.layers['log_background_normalized_zeroed'] = adata.layers['log_background_normalized'].copy()
        # Zero out negative values
        adata.layers['log_background_normalized_zeroed'][adata.layers['log_background_normalized_zeroed'] < 0] = 0
        
        adata.obs['TissueID']= adata.obs['imageid'].str.split('_').str[0]
        
        adata_list.append(adata)
    else:
        print(f"Skipping {case_number}: No gates.csv file found, cannot normalize and combine")
        


Reading in raw adata for Case1-31
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv
Subtracting the following gate values from each marker:
  Hoechst_1: 0.0
  PRAME: 2.86
  MITF: 4.3
  JUND: 3.7
  SOX10: 4.6
  cJUN: 2.9
  cFOS: 3.9
  NGFR: 6.5
  SOX9: 4.2
  Ki67: 2.85
  CD45: 3.6
  AXL: 3.6
  FRA1: 4.9
  Hoechst_6: 0.0
  CD4: 3.9
  JUNB: 2.9
  pancK: 2.7
  FRA2: 3.0
  CD163: 4.76
  alphaSMA: 3.4
  CD8: 4.1
  CD19: 5.08
  Hoechst_11: 7.343920451148872
  CD31: 4.1
  PDGFRBeta: 4.86
  pSTAT1: 3.48
  FOXP3: 3.7
  CD68: 3.4
  BRAFV600E: 5.0
  phospho-p38: 3.7
  anti-Melanoma: 2.87
  phospho-p44: 5.71
  Hoechst_17: 7.251324716064622
  FAP: 5.21
Background normalization complete. Layer 'log_background_normalized' added.
Reading in raw adata for Case2-8


KeyboardInterrupt: 

In [12]:
adata_list

[AnnData object with n_obs × n_vars = 33720 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw', 'log_background_normalized', 'log_background_normalized_zeroed']

In [21]:
for adata in adata_list:
    adata.obs_names = adata.obs_names.str.replace(r'Case1-31_feature-extraction_merged', '', regex=True)

In [22]:
adata_list= []
for case in case_numbers:
    adata_path = os.path.join(project_dir, 'raw_adata', f'{case}_raw.h5ad')
    adata = ad.read_h5ad(adata_path)
    
    adata_list.append(adata)

In [29]:
processed_adata_list= []
for case in case_numbers:
    adata_path = os.path.join(project_dir, 'processed_adata', f'{case}_adata.h5ad')
    adata = ad.read_h5ad(adata_path)
    
    processed_adata_list.append(adata)

In [65]:
for adata in processed_adata_list:
    adata.obs_names = adata.obs_names.str.replace(r'_feature-extraction_merged', '', regex=True)

In [ ]:
for adata in adata_list:
    adata.obs['TissueID'] = adata.obs['imageid'].str.split('_').str[0]
    adata.obs_names = adata.obs_names.str.replace(r'_feature-extraction_merged', '', regex=True)
    

In [41]:
adata_list[2].obs['imageid']="Case3-9"

In [43]:
adata_list[2].obs['imageid']

Case3-9_1         Case3-9
Case3-9_2         Case3-9
Case3-9_3         Case3-9
Case3-9_4         Case3-9
Case3-9_5         Case3-9
                   ...   
Case3-9_601477    Case3-9
Case3-9_601478    Case3-9
Case3-9_601479    Case3-9
Case3-9_601480    Case3-9
Case3-9_601481    Case3-9
Name: imageid, Length: 601481, dtype: object

In [64]:
for adata in adata_list[2:3]:
    case_number= "Case3-9"
    print(f"Adding ROI image for {case_number}")
    image_search_pattern = os.path.join(project_dir, case_number, f'*{case_number}*fullpanel*tif')
    image_files = glob.glob(image_search_pattern, recursive=True)
    
    if image_files:
        image_path = image_files[0]
        print(f"image_path for {case_number}: {image_path}")
    else:
        image_path = None
        print(f"No image found for {case_number}")
        
    sm.pl.addROI_image(image_path, adata, markers=['Hoechst_1','MITF','SOX10','PRAME','NGFR','SOX9','cFOS','cJUN','CD45'])
    print(adata.obs['ROI'].value_counts(dropna=False))
    ROIs = adata.obs['ROI']
    print(f"Saving ROIs for {case_number}")
    ROIs.to_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index=True)

Adding ROI image for Case3-9
image_path for Case3-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_fullpanel_final.tif

        Opening Napari;
        Add shape layers (on left) to draw ROI's. 
        Rename the shape layer to give a name to your ROI
        Multiple shape layers are supported
        ROI's should not overlap
        Close Napari to save ROI's.
        


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/napari/_vispy/layers/image.py:251: UserWarning:

data shape (22700, 25500) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/napari/_vispy/layers/image.py:251: UserWarning:

data shape (22700, 25500) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.



Identifying cells within selected ROI's
ROIs saved under adata.obs['ROI']
ROI
Tumor    504129
Other     97352
Name: count, dtype: int64
Saving ROIs for Case3-9


if you need to, load in ROI labels from csv

In [20]:
for adata in adata_list:
    case_number=str(adata.obs['imageid'].unique()[0])
    ROI_path = glob.glob(os.path.join(project_dir, case_number, '*ROIs.csv'))[0]
    ROIs= pd.read_csv(ROI_path, index_col=0)
    adata.obs['ROI'] = ROIs

IndexError: list index out of range

In [48]:
processed_adata_list

[AnnData object with n_obs × n_vars = 33720 × 17
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI', 'phenotype', 'phenotype_category'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw', 'log_background_normalized', 'log_background_normalized_zeroed',
 AnnData object with n_obs × n_vars = 674801 × 17
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI', 'phenotype', 'phenotype_category'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw', 'log_background_normalized', 'log_background_normalized_zeroed',
 AnnData object with n_obs × n_vars = 61028 × 17
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID'

In [49]:
adata_list

[AnnData object with n_obs × n_vars = 33720 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID'
     uns: 'all_markers'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 674801 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID'
     uns: 'all_markers'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 601481 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI'
     uns: 'all_markers'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 61028 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Exte

In [66]:
for case_number, adata in zip(case_numbers, processed_adata_list):
    ROIs = adata.obs['ROI']
    print(f"Saving ROIs for {case_number}")
    ROIs.to_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index=True)

Saving ROIs for Case1-31
Saving ROIs for Case2-8
Saving ROIs for Case4-5_R1
Saving ROIs for Case4-5_R2
Saving ROIs for Case7-10
Saving ROIs for Case8-9_R1
Saving ROIs for Case8-9_R2
Saving ROIs for Case9-7_R2
Saving ROIs for Case10-3_R1
Saving ROIs for Case10-3_R2
Saving ROIs for Case11-13


In [54]:
len(adata_list)

12

In [68]:
for case_number, adata in zip(case_numbers, adata_list):
    
    ROIs= pd.read_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index_col=0)
    adata.obs['ROI'] = ROIs

In [70]:
adata_list[2].obs['ROI'].value_counts()

ROI
Tumor    504129
Other     97352
Name: count, dtype: int64

In [81]:

gate_file_paths = {}

for case_number in case_numbers:
    # Search for gates.csv inside the case-specific subdirectory
    search_path = os.path.join(project_dir, case_number, '*gates.csv')
    matches = glob.glob(search_path)
    if matches:
        gate_file_paths[case_number] = matches[0]  # Take the first match
        print(f"Found for {case_number}: {matches[0]}")
    else:
        gate_file_paths[case_number] = None
        print(f"No gates.csv file found for {case_number}")

Found for Case1-31: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv
Found for Case2-8: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case2-8/Case2-8_full_gates.csv
Found for Case3-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_full_gates.csv
Found for Case4-5_R1: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R1/Case4-5_R1_full_gates.csv
Found for Case4-5_R2: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R2/Case4-5_R2_full_gates.csv
Found for Case7-10: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case7-10/Case7-10_full_gates.csv
Found for Case8-9_R1: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analy

In [85]:
for case_number,adata in zip(case_numbers,adata_list):
    print(f"Copying gates from {gate_file_paths[case_number]}")
    marker_gates = pd.read_csv(os.path.join( project_dir, case_number, f'{case_number}_full_gates.csv'),
    header = [0], index_col =[0])
    adata.uns['gates'] = marker_gates

Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case2-8/Case2-8_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R1/Case4-5_R1_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case4-5_R2/Case4-5_R2_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case7-10/Case7-10_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/2025070

## Phenotyping

In [89]:
adata_list[1].uns['gates']

,Case2-8
Hoechst_1,8.064459
PRAME,3.400000
MITF,3.280000
JUND,4.100000
SOX10,5.540000
cJUN,4.060000
cFOS,4.290000
NGFR,5.420000
SOX9,4.110000
Ki67,4.580000


Loop through adata list, rescale data according to gates, and run phenotyping. Data is shifted so the gate is the new 0; positive values will be phenotyped as positive and negative values will be phenotyped as negative. If it's running into an error where it can't find gates, check adata.uns['gates']. Column name for the gates should match imageid

In [99]:
for adata in adata_list:
    adata.obs['imageid']=adata.obs['imageid'].str.split('_').str[0]

In [100]:
adata_list[0].obs['imageid']

Case1-31_1        Case1-31
Case1-31_2        Case1-31
Case1-31_3        Case1-31
Case1-31_4        Case1-31
Case1-31_5        Case1-31
                    ...   
Case1-31_33716    Case1-31
Case1-31_33717    Case1-31
Case1-31_33718    Case1-31
Case1-31_33719    Case1-31
Case1-31_33720    Case1-31
Name: imageid, Length: 33720, dtype: object

In [103]:
adata_list

[AnnData object with n_obs × n_vars = 33720 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 674801 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 601481 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw',
 AnnData object with n_obs × n_vars = 61028 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisL

In [104]:
for adata in adata_list:
    # Replace NaN in expression matrix
    if isinstance(adata.X, np.ndarray):
        adata.X = np.nan_to_num(adata.X, nan=0)
    else:
        # if it's a sparse matrix
        adata.X.data = np.nan_to_num(adata.X.data, nan=0)

    # Replace NaN in adata.obs and adata.var
    adata.obs = adata.obs.fillna(0)
    adata.var = adata.var.fillna(0)

/var/folders/vb/f8_hhkls73qf5938jf5w6rk9dds1_2/T/ipykernel_80360/3835958844.py:10: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/vb/f8_hhkls73qf5938jf5w6rk9dds1_2/T/ipykernel_80360/3835958844.py:10: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/vb/f8_hhkls73qf5938jf5w6rk9dds1_2/T/ipykernel_80360/3835958844.py:10: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_

In [106]:
print(adata_list[0].var_names)
print("NaN count per marker:", np.isnan(adata_list[0].X).sum(axis=0))

Index(['Hoechst_1', 'PRAME', 'MITF', 'JUND', 'SOX10', 'cJUN', 'cFOS', 'NGFR',
       'SOX9', 'Ki67', 'CD45', 'AXL', 'FRA1', 'Hoechst_6', 'CD4', 'JUNB',
       'pancK', 'FRA2', 'CD163', 'alphaSMA', 'CD8', 'CD19', 'Hoechst_11',
       'CD31', 'PDGFRBeta', 'pSTAT1', 'FOXP3', 'CD68', 'BRAFV600E',
       'phospho-p38', 'anti-Melanoma', 'phospho-p44', 'Hoechst_17', 'FAP'],
      dtype='object')
NaN count per marker: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [107]:
adata_list[0].uns['gates']

,markers,gates
0,AXL,3.600000
1,CD4,3.900000
2,CD45,3.600000
3,FRA1,4.900000
4,FRA2,3.000000
5,Hoechst,5.000000
6,JUNB,2.900000
7,JUND,3.700000
8,Ki67,2.850000
9,MITF,4.300000


In [110]:
adata_list[2].X

array([[7.11824365, 3.6653594 , 3.28538609, ..., 3.19831502, 6.63189776,
        3.17007039],
       [7.27709414, 1.87180218, 2.72639934, ..., 3.56718252, 6.7360439 ,
        5.79932573],
       [8.38213795, 2.66600196, 2.05525432, ..., 3.36170212, 8.0901048 ,
        3.78086974],
       ...,
       [7.97372321, 2.88210725, 4.04952797, ..., 4.85765677, 7.2900023 ,
        5.40430534],
       [8.00921074, 3.8867052 , 3.95524212, ..., 7.72341503, 7.62863921,
        4.2645153 ],
       [7.60739522, 3.22992566, 3.25916434, ..., 7.19581228, 6.64353652,
        2.81062202]])

In [109]:
adata_list[2].uns['gates']

,Case3-9
markers,
Hoechst_1,7.766028
PRAME,3.350000
MITF,3.300000
JUND,3.750000
SOX10,5.500000
cJUN,3.170000
cFOS,3.600000
NGFR,5.250000
SOX9,4.200000


In [108]:
sm.pp.rescale(adata_list[2], gate=adata_list[2].uns['gates'])

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:145: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




Scaling Image: Case3-9
Scaling Hoechst_1 (gate: 7.766)
Scaling PRAME (gate: 3.350)
Scaling MITF (gate: 3.300)
Scaling JUND (gate: 3.750)
Scaling SOX10 (gate: 5.500)
Scaling cJUN (gate: 3.170)
Scaling cFOS (gate: 3.600)
Scaling NGFR (gate: 5.250)


ValueError: An exception occurred: NGFR has nan values

In [ ]:
for i in range(len(adata_list)):
    adata = adata_list[i]

    # Validate gate format if necessary (optional safety check)
    

    sm.pp.rescale(adata, gate=adata.uns['gates'])

    adata = sm.tl.phenotype_cells(adata, phenotype=phenotype, label="phenotype")

    adata_list[i] = adata  # store back if needed

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Running GMM for image: Case1-31
Applying GMM to markers: Hoechst_1, Hoechst_6

Scaling Image: Case1-31
Scaling Hoechst_1 (gate: 7.753)
Scaling PRAME (gate: 2.860)
Scaling MITF (gate: 4.300)
Scaling JUND (gate: 3.700)
Scaling SOX10 (gate: 4.600)
Scaling cJUN (gate: 2.900)
Scaling cFOS (gate: 3.900)
Scaling NGFR (gate: 6.500)


ValueError: An exception occurred: NGFR has nan values

Replace all "Tumor cell" phenotypes with "Unknown"

In [ ]:
for i in range(len(adata_list)):
    adata = adata_list[i]
    adata.obs['phenotype'] = adata.obs['phenotype'].replace('Tumor cell', 'Unknown')


Assign cell phenotype category. ie. Identify tumor cells vs TME 

In [ ]:
for i in range(len(adata_list)):
    adata = adata_list[i]
    adata.obs['phenotype_category'] = adata.obs['phenotype'].apply(hm.categorize_phenotype)

In [111]:
for adata in adata_list:
    adata.write_h5ad(os.path.join(project_dir,'processed_adata',f"{adata.obs['imageid'].unique()[0]}_TME_new_adata.h5ad"))